# MLFlow-based Experiment

In [1]:
!pip install mlflow boto3

In [2]:
import os
import requests
import mlflow

In [ ]:
# configuration parameters (applied through papermill)
aws_access_key_id = 'access_key_goes_here'
aws_secret_key = 'secret_key_goes_here'
aws_default_region = 'lab'
mlflow_s3_endpoint_url = 'https://eclipse.home.glroland.com:9000'
mlflow_s3_ignore_tls = 'true'

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key
os.environ['AWS_DEFAULT_REGION'] = aws_default_region
os.environ['MLFLOW_S3_ENDPOINT_URL'] = mlflow_s3_endpoint_url
os.environ['MLFLOW_S3_IGNORE_TLS'] = mlflow_s3_ignore_tls

print (os.environ)

In [4]:
mlflow.set_tracking_uri(uri="http://mlflow.home.glroland.com:8080")

In [5]:
experiment_name = "baseball_mlfow_v1"
if mlflow.get_experiment_by_name(experiment_name) is None:
    print ("Creating new experiment: " + experiment_name)
    mlflow.create_experiment(experiment_name, artifact_location='s3://mlflow')
else:
    print ("Experiment already exists: " + experiment_name)
mlflow.set_experiment(experiment_name)

Experiment already exists: baseball_mlfow_v1


<Experiment: artifact_location='s3://mlflow', creation_time=1738683213822, experiment_id='318457801545587060', last_update_time=1738683213822, lifecycle_stage='active', name='baseball_mlfow_v1', tags={}>

In [6]:
mlflow.start_run()

<ActiveRun: >

In [7]:
# Log a parameter (key-value pair)
mlflow.log_param("hello", "world")

# Log a metric; metrics can be updated throughout the run
mlflow.log_metric("foo", 1)
mlflow.log_metric("foo", 2)
mlflow.log_metric("foo", 3)

# Log an artifact (output file)
output_filename = "/tmp/output.txt"
with open(output_filename, "w") as f:
    f.write("Hello world!")
mlflow.log_artifact(output_filename)



/opt/anaconda3/envs/kfp/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eclipse.home.glroland.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
model_name = "model.onnx"
url = "https://eclipse.home.glroland.com:9000/baseball/play/1/" + model_name
temp_file = "/tmp/" + model_name

# stream content
response = requests.get(url, stream=True)
response.raw.decode_content = True

# save file
with open(temp_file, 'wb') as f:
    for chunk in response.iter_content(chunk_size=1024): 
        f.write(chunk)

# log artifact
mlflow.log_artifact(temp_file, artifact_path="1")

/opt/anaconda3/envs/kfp/lib/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eclipse.home.glroland.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
run = mlflow.active_run()
s3_url = run.info.artifact_uri

print(f"Artifact URI (pre-processing): {s3_url}")

#"https://eclipse.home.glroland.com:9000/baseball/play/1/model.onnx"

s3_url = s3_url.replace("s3:/", os.environ['MLFLOW_S3_ENDPOINT_URL'])
s3_url += "/1/" + model_name

print(f"Artifact URI (post-processing): {s3_url}")


Artifact URI (pre-processing): s3://mlflow/3e5135617a374ee9b3a06313ca08436b/artifacts
Artifact URI (post-processing): https://eclipse.home.glroland.com:9000/mlflow/3e5135617a374ee9b3a06313ca08436b/artifacts/1/model.onnx


In [10]:
mlflow.end_run()

🏃 View run stylish-bug-129 at: http://mlflow.home.glroland.com:8080/#/experiments/318457801545587060/runs/3e5135617a374ee9b3a06313ca08436b
🧪 View experiment at: http://mlflow.home.glroland.com:8080/#/experiments/318457801545587060
